# Reading Trajectory Data From Files

In [ ]:
import pandas as pd
from geopandas import GeoDataFrame, read_file

import sys
sys.path.append("..")
import movingpandas as mpd
print(mpd.__version__)

import warnings
#warnings.simplefilter("ignore")

## Geopackage

### with DatetimeIndex

In [ ]:
%%time
gdf = read_file('data/demodata_geolife.gpkg')
gdf['t'] = pd.to_datetime(gdf['t'])
gdf = gdf.set_index('t').tz_localize(None)
traj_collection = mpd.TrajectoryCollection(gdf, 'trajectory_id')
print(traj_collection)

### without DatetimeIndex

In [ ]:
%%time
gdf = read_file('data/demodata_geolife.gpkg')
traj_collection = mpd.TrajectoryCollection(gdf, 'trajectory_id', t='t')
print(traj_collection)

In [ ]:
%%time
gdf = read_file('data/demodata_ais.gpkg')
gdf['t'] = pd.to_datetime(gdf['Timestamp'], format='%d/%m/%Y %H:%M:%S')
gdf = gdf[gdf.SOG>0]
traj_collection = mpd.TrajectoryCollection(gdf, 'MMSI', min_length=100, t='t')
print(traj_collection)

## CSV

In [ ]:
%%time 
df = pd.read_csv('data/demodata_geolife.csv', delimiter=';')
traj_collection = mpd.TrajectoryCollection(df, 'trajectory_id', t='t', x='X', y='Y')
traj_collection.hvplot(title=str(traj_collection), line_width=7.0, tiles='StamenTonerBackground', frame_width=700, frame_height=400)

## Error Messages

### Missing datetime info

In [ ]:
gdf = read_file('data/demodata_geolife.gpkg')
traj_collection = mpd.TrajectoryCollection(gdf, 'trajectory_id')

### Missing geometry info

In [ ]:
df = pd.read_csv('data/demodata_geolife.csv', delimiter=';')
traj_collection = mpd.TrajectoryCollection(df, 'trajectory_id', t='t')

## Continue exploring MovingPandas

1. [Getting started](1-getting-started.ipynb)
1. [Reading data from files](2-reading-data-from-files.ipynb)
1. [Trajectory aggregation (flow maps)](3-generalization-and-aggregation.ipynb)
1. [Stop detection](4-stop-detection.ipynb)